In [1]:
import pickle
import math
import itertools
import scipy.stats
import numpy as np

In [20]:
M = 30
N = 20
R = 10
A = [[0.25,0.75],[0.75,0.25]]

In [72]:
def load_obj(name ):
    with open('./' + name + '.pkl', 'rb') as f:
        return pickle.load(f, encoding='latin1')
    
sequence_outputs = load_obj("sequence_output")



data_dict ={}
for key in sorted(sequence_outputs.keys()):
    data_dict[key] = sequence_outputs[key]

pairs = list(data_dict.keys())

In [11]:
import pickle
import random
import itertools
import numpy as np
from collections import defaultdict


#Variables you need
M = 30
N = 20
R = 10
beta = np.pi
sigma = 5
# mean for each subfield with respect to the player
mu = np.array([np.random.normal(10, sigma, (M,N)), 
               np.random.normal(23, sigma, (M,N))])
print ("Mean matrix shape: ",mu.shape)

print(mu)
#Function to save dictionary in a pickle file
def save_obj(obj, name ):
    with open('./'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        

def generate_data():            
    '''
    Function to generate tournament data
    '''
    # Generate Players List and list of pairs
    players_list = [ i+1 for i in range (N)]
    player_pairs = list(itertools.combinations(players_list, 2))
    output_sequences = defaultdict(dict)

    for player_pair in player_pairs:
        for i in range(R):
            output_sequences[player_pair][i+1] = []
            sequence = []
            table_idx = 0     #If we are in left side of M or right side
            for j in range(M):
                choice = np.random.choice(['switch', 'stay'], p=[3/float(4), 1/float(4)])
                if choice == 'switch':
                    table_idx = 1 - table_idx
                player1_performance = np.random.normal(mu[table_idx,j,player_pair[0]-1], beta, 1)
                player2_performance = np.random.normal(mu[table_idx,j,player_pair[1]-1], beta, 1)
                sequence.append(player1_performance+player2_performance)

            output_sequences[player_pair][i+1] = sequence
    return output_sequences

#if __name__ == '__main__':
    
output_sequences = generate_data()

Mean matrix shape:  (2, 30, 20)
[[[ 9.09704858  8.2172244   7.01190022 ...  5.17241157  1.32388834
    9.47803901]
  [10.77719208  8.6577554  14.5673643  ...  8.2255238   7.41352843
   -0.99091372]
  [ 7.98440287 15.31955413 10.52138635 ... 15.69936471  6.36954202
    4.7943845 ]
  ...
  [ 8.31837519 22.68187853 13.16779578 ... 14.64381904 15.31280975
   15.93020098]
  [14.63232346  8.6752246   9.68192368 ... 13.59641344  7.41358396
    9.07229364]
  [13.7657913  10.92633266 13.34848841 ...  1.37827788 12.35161034
   10.60821523]]

 [[25.43394719 23.61637391 17.09862633 ... 16.88327035 26.71628803
   28.95026354]
  [29.12626172 30.84553524 26.54444712 ... 27.56605679 23.006448
   17.4481759 ]
  [17.76515842 25.21490164 20.20358107 ... 25.53721955 27.10379796
   20.58872351]
  ...
  [28.47276739 19.46784011 37.37116889 ... 26.42981037 20.45898196
   26.69366125]
  [17.78027023 27.06907839 20.11319162 ... 11.08996945 29.48941266
   21.75026971]
  [20.27944177 20.77256278 15.14532818 ... 

In [12]:
data_dict ={}
for key in sorted(output_sequences.keys()):
    data_dict[key] = output_sequences[key]

pairs = list(data_dict.keys())

In [13]:
orginal_MU = mu

In [14]:
X = np.zeros([N,N,R,M])
for n in range(N):
    for m in range(N):
        if n == m:
            continue
        for r in range(R):
            for s in range(M):
                if (n+1,m+1) in pairs:
                    X[n,m,r,s] = data_dict[(n+1,m+1)][r+1][s]
                else:
                    X[n,m,r,s] = data_dict[(m+1,n+1)][r+1][s]

In [15]:
# Intialize

In [45]:
# means for all pairs for all subfields
sigma = 5
mu = np.array([np.random.normal(15, sigma, (M,N)), 
               np.random.normal(20, sigma, (M,N))]).T
beta = 5

In [46]:
def calB(mu, beta):
    blist = []
    B = np.ones([N,N,R,M,2])
    for n in range(N):
        for m in range(N):
            if n == m or (n,m) in blist: 
                continue
            else:
                #for r in range(R):
                for s in range(M):
                    B[n,m,:,s] = scipy.stats.norm((mu[n]+mu[m])[s], beta*np.sqrt(2)).pdf(np.reshape(X[n,m,:,s], [R,1]))
                    B[m,n,:,s] = B[n,m,:,s]
            blist.append((n,m))
            blist.append((m,n))
    return B

def get_transition(r1, r2):
    # calculate A((r1, m1), (r2, m1+1)) (for test purpose we set below)
    A = [[0.25,0.75],[0.75,0.25]]
    return A[r1][r2]

def get_emission(n, m, r, s, o, B):
    # calculate O(m, o) (for test purpose we set below)
    O = B[n,m,r,s,o]
    return O

def get_init():
    # provide an array containing the initial state probability having size R (for test purpose we set below)
    pi = np.array([0.25, 0.75])
    # number of rows
    R = pi.shape[0]
    return pi, R

def forward(n, q, t, get_init, get_transition, get_emission, M, B):
    pi, K = get_init()
    alpha = np.zeros((M, K))

    # base case
    O = []
    for r in range(K):
        O.append(get_emission(n, q, t, 0, r, B))
    alpha[0, :] = pi * O[:]

    # recursive case
    for m in range(1, M):
        for r2 in range(K):
            for r1 in range(K):
                #print(n,q,t,m,r2)
                transition = get_transition(r1, r2)
                emission = get_emission(n, q, t, m, r2, B)
                alpha[m, r2] += alpha[m - 1, r1] * transition * emission
    return alpha, np.sum(alpha[M - 1, :])


def backward(n, q, t, get_init, get_transition, get_emission, M, B):
    pi, K = get_init()
    beta = np.zeros((M, K))
    # base case
    beta[M - 1, :] = 1

    # recursive case
    for m in range(M - 2, -1, -1):
        for r1 in range(K):
            for r2 in range(K):
                transition = get_transition(r1, r2)
                emission = get_emission(n, q, t, m+1, r2, B)
                beta[m, r1] += beta[m + 1, r2] * transition * emission
    O = []
    for r in range(K):
        O.append(get_emission(n, q, t, 0, r, B))

    return beta, np.sum(pi * O[:] * beta[0, :])

def cal_gammas(n,q,l,B):
    alpha, ha = forward(n,q,l, get_init, get_transition, get_emission, M, B)
    beta, _ = backward(n,q,l, get_init, get_transition, get_emission, M, B)
    gamma = alpha*beta/ha
    digamma = np.zeros([M,2,2])
    for t in range(M-1):
        for i in range(2):
            for j in range(2):
                digamma[t,i,j] = alpha[t,i]*A[i][j]*beta[t+1,j]*B[n,q,l,t+1,j]
        digamma[t] = digamma[t]/np.sum(digamma[t])
    return gamma, digamma

def all_gammas(B):
    gamma_all = np.zeros([N,N,R,M,2])
    digamma_all = np.zeros([N,N,R,M,2,2])
    for n in range(N):
        for m in range(N):
            if m == n: continue
            for r in range(R):
                gamma_all[n,m,r], digamma_all[n,m,r] = cal_gammas(n,m,r,B)
    return gamma_all, digamma_all

def Qfunc(gamma_all, digamma_all, B, A):
    return (np.sum(gamma_all*np.log(B))+np.sum(digamma_all*np.log(A)))/2.0

def re_estimate_mu(gamma_all, mu):
    mu_new = np.zeros_like(mu)
    for n in range(N):
        for t in range(M):
            for k in range(2):
                x = 0
                h = 0
                for m in range(N):
                    if m == n: continue
                    y = 0
                    g = 0
                    for r in range(R):
                        y += gamma_all[n,m,r,t,k]*(X[n,m,r,t]-mu[m,t,k])
                        g += gamma_all[n,m,r,t,k]
                    x += y
                    h += g
                mu_new[n,t,k] = x/h
    return mu_new

def re_estimate_beta(gamma_all, mu):
    bnew = np.zeros([N,N,R,M,2])
    for n in range(N):
        for m in range(N):
            for r in range(R):
                for t in range(M):
                    bnew[n,m,r,t] = gamma_all[n,m,r,t]*(X[n,m,r,t]-mu[n,t]-mu[m,t])**2
    bet = np.sqrt(np.sum(bnew)/(2*np.sum(gamma_all)))
    return bet


In [47]:
def train(mu, beta, oldVal): 
    B = calB(mu, beta)
    gamma_all, digamma_all = all_gammas(B)
    val = Qfunc(gamma_all, digamma_all, B, A)
    if val > oldVal + 1:
        oldVal = val
        mu_new = re_estimate_mu(gamma_all, mu)
        beta_new = re_estimate_beta(gamma_all, mu)
        print(val, beta)
        return train(mu_new, beta_new, oldVal)
    else:
        print("Converged: ", val, beta)
        return mu, beta

In [48]:
oldVal = -np.Inf
m, b = train(mu, beta, oldVal)

-298451.82296046196 5
-247593.43226258902 9.468045150015776
-237600.2843586056 7.427268129529785
-222818.7269808112 6.426169308683692
-209344.75650731847 4.602935178225618
-202160.59499349468 3.8191840508386288
-200553.24678932843 3.4013406861296
-199754.74041864404 3.3438582437169178
-199741.1999355676 3.290625736937127
-198671.3130190007 3.2985716443781588
-198345.79727005571 3.2305162405165087
-197805.41892585784 3.216273138534212
Converged:  -197952.3653821977 3.1798530073158493


In [49]:
m.shape

(20, 30, 2)

In [50]:
b

3.1798530073158493